In [1]:
import numpy as np
import cupy as cp
from holotomocupy.chunking import Chunking
np.random.seed(10)
a = np.random.random([15,64,64])
b = np.random.random([15,64,64])

nchunk = 4
ngpus = 3
nbytes = 64*64*nchunk*64 

class mClass():
    def __init__(self,nbytes,ngpus):        
        self.cl_chunking = Chunking(nbytes, nchunk, ngpus)
        self.ngpus = ngpus
    
    def msum(self,a,b,d,c):
        out0 = np.zeros_like(a.swapaxes(0,1))
        out00 = np.zeros_like(a.swapaxes(0,1))
        out1 = [None]*self.ngpus
        out2 = [None]*self.ngpus
        for i in range(ngpus):
            with cp.cuda.Device(i):
                out1[i] = cp.zeros([1],dtype='float64')
                out2[i] = cp.zeros([64,1,64],dtype='float64')

        @self.cl_chunking.gpu_batch(axis_inp=0,axis_out=1,nout=4)
        def _msum(self,out0,out00,out1,out2,a,b,d,c):
            out0[:] = (a+b+d*c).swapaxes(0,1)
            out00[:] = (a-b).swapaxes(0,1)
            out1[:] += cp.linalg.norm(a-b)**2
            out2[:] += (cp.linalg.norm(a+b,axis=0)**2)[None].swapaxes(0,1)

        _msum(self,out0,out00,out1,out2,a,b,d,c)
        out1 = sum(out1[1:],out1[0])[0].get()
        out2 = sum(out2[1:],out2[0]).get()
        return out0,out00,out1,out2

cl = mClass(nbytes,ngpus)

d = cp.random.random(a.shape)[0:1]
o1,o11,o2,o22 = cl.msum(a,b,d,2)

print(a.shape,b.shape,d.shape)
oo1 = (a+b+2*d.get()).swapaxes(0,1)
oo11 = (a-b).swapaxes(0,1)
oo2 = np.linalg.norm(a-b)**2
oo22 = (np.linalg.norm(a+b,axis=0)**2)[None].swapaxes(0,1)
print(np.linalg.norm(o1),np.linalg.norm(oo1),np.linalg.norm(o1-oo1))
print(np.linalg.norm(o11),np.linalg.norm(oo11),np.linalg.norm(o11-oo11))
print(np.linalg.norm(o2),np.linalg.norm(oo2),np.linalg.norm(o2-oo2))
print(np.linalg.norm(o22),np.linalg.norm(oo22),np.linalg.norm(o22-oo22))




(15, 64, 64) (15, 64, 64) (1, 64, 64)
519.7316594803392 519.7316594803392 0.0
101.21252862734524 101.21252862734524 0.0
10243.975951141176 10243.97595114118 3.637978807091713e-12
1135.3970452032154 1135.3970452032154 2.2216181354152164e-13


/tmp/ipykernel_3672967/613224400.py:35: PerformanceWarning: The device where the array resides (1) is different from the current device (0). Peer access has been activated automatically.
  out1 = sum(out1[1:],out1[0])[0].get()
/tmp/ipykernel_3672967/613224400.py:35: PerformanceWarning: The device where the array resides (2) is different from the current device (0). Peer access has been activated automatically.
  out1 = sum(out1[1:],out1[0])[0].get()
/tmp/ipykernel_3672967/613224400.py:36: PerformanceWarning: The device where the array resides (1) is different from the current device (0). Peer access has been activated automatically.
  out2 = sum(out2[1:],out2[0]).get()
/tmp/ipykernel_3672967/613224400.py:36: PerformanceWarning: The device where the array resides (2) is different from the current device (0). Peer access has been activated automatically.
  out2 = sum(out2[1:],out2[0]).get()


: 